This is an MPCP solver, in python3.   The problem is not decidable in general, so the integer parameter B controls how long we are willing to search for a match, before giving up.

The first code cell defines the solver (a lot of code), it is easier to skip that and look at the usage examples below.

In [0]:
# A 'domino' is a pair of strings, as described by Sipser.

# Helper functions
def reduce(a, b):
  # Return prefix-reduced form of strings a and b, or None.
  n = min(len(a), len(b))
  return None if a[:n]!=b[:n] else (a[n:], b[n:])

def seq2list(seq):
  # Convert (3, (2, (1, None))) to [1, 2, 3]
  ret = []
  while seq != None:
    ret.append(seq[0])
    seq = seq[1]
  return ret[::-1]

# We use a priority queue, to prefer exploring reduced
# pairs which are close to the goal (short total length)
import heapq
def pqpair(r):
  return (len(r[0])+len(r[1]), r)

# Main function (no printed output)
def mpcp(dominos, B=10000):
  """
  Given a list of dominos, try to find an MPCP match.
  That is, a match starting with a copy of dominos[0].
  If found, the match is returned as a list of indices.
  The integer B bounds the number of steps searched.
  If there is no match, returns None.
  If we ran out of steps, returns B.
  """
  pq, seq = [], {}
  start = reduce(dominos[0][0], dominos[0][1])
  if start:
    heapq.heappush(pq, pqpair(start))
    seq[start] = (0, None)
  goal = ('', '')
  for step in range(B):
    if len(pq)==0: # no solution!
      return None
    thispair = heapq.heappop(pq)[1]
    thisseq = seq[thispair]
    if thispair == goal: # found it!  return match
      return seq2list(thisseq)
    # Try extending thispair with each domino
    for i in range(len(dominos)):
      dom = dominos[i]
      r = reduce(thispair[0]+dom[0], thispair[1]+dom[1])
      if r and (r not in seq): # new reduced pair
        seq[r] = (i, thisseq)
        if r==goal: # found it!
          return seq2list(seq[r])
        # save r for later
        heapq.heappush(pq, pqpair(r))
  # ran out of steps
  return B

def mpcp_pretty(dominos, B=10000):
  "Run mpcp(dominos, B), but with printed output."
  print('Solving MPCP:', 
        ' '.join("%s/%s" % d for d in dominos))
  sol = mpcp(dominos, B)
  if sol==None:
    print("NO MATCH (no solution exists)")
  elif sol==B:
    print("NO MATCH FOUND (gave up after %d steps)" % B)
  else:
    print("FOUND MATCH:", ' '.join(str(i) for i in sol))
    print(' '.join(dominos[i][0] for i in sol))
    print(' '.join(dominos[i][1] for i in sol))
  print()

Run some examples

In [0]:
# Example from Sipser page 227
page227 = [('b', 'ca'), ('a', 'ab'), ('ca', 'a'), ('abc', 'c')]
mpcp_pretty(page227) # no solution

# Rearraged, moved the second domino to the front
page227b = [('a', 'ab'), ('b', 'ca'), ('ca', 'a'), ('abc', 'c')]
mpcp_pretty(page227b) # finds match in book

# Exercise 5.3, Sipser page 239
page239 = [('ab', 'abab'), ('b', 'a'), ('aba', 'b'), ('aa', 'a')]
mpcp_pretty(page239)  # finds match in book

Solving MPCP: b/ca a/ab ca/a abc/c
NO MATCH (no solution exists)

Solving MPCP: a/ab b/ca ca/a abc/c
FOUND MATCH: 0 1 2 0 3
a b ca a abc
ab ca a ab c

Solving MPCP: ab/abab b/a aba/b aa/a
FOUND MATCH: 0 0 2 1 1 3 3
ab ab aba b b aa aa
abab abab b a a a a



In [0]:
# Examples with no solution

mpcp_pretty([('b', 'baaab'), ('aa', '')]) # no solution

mpcp_pretty([('b', 'baab'), ('aa', '')]) # gives up

Solving MPCP: b/baaab aa/
NO MATCH (no solution exists)

Solving MPCP: b/baab aa/
NO MATCH FOUND (gave up after 10000 steps)



In [8]:
# First pick x, an input string in {a,b}*
x = 'aabbaba'

# Next, we define a list of dominos.
# Each domino is a pair of strings.
# We have a copy of x in the first domino,
# but all the remaining dominos are fixed.

dominos = [
  ('#', '#S'+x+'#'), 
  ('a', 'a'), ('b', 'b'), ('#', '#'),
  ('Sa', 'S'), ('Sb', 'T'),
  ('Ta', 'T'), ('Tb', 'S'),
  ('#T#', '#')
]

# try to find a match
mpcp_pretty(dominos)

# Define language L1 = { w in {a,b}*: w has an odd number of b's }.
#
# Claim: there is a match iff x is in L1.

Solving MPCP: #/#Saabbaba# a/a b/b #/# Sa/S Sb/T Ta/T Tb/S #T#/#
FOUND MATCH: 0 4 1 2 2 1 2 1 3 4 2 2 1 2 1 3 5 2 1 2 1 3 7 1 2 1 3 4 2 1 3 5 1 3 6 8
# Sa a b b a b a # Sa b b a b a # Sb b a b a # Tb a b a # Sa b a # Sb a # Ta #T#
#Saabbaba# S a b b a b a # S b b a b a # T b a b a # S a b a # S b a # T a # T #

